# Faces scenario

This notebook creates a new scenario where the face of the person is the only thing visible in the video.

In [ ]:
import cv2
import numpy as np
import respiration.roi as roi


def get_face_masks(frames):
    """This function returns the segmentation masks of the face of the person in the video."""
    masks = []
    for frame in frames:
        faces = roi.detect_faces(frame, scale_factor=1.5)
        roi_mask = np.zeros_like(frame)

        for face in faces:
            mask = roi.mask_from_area(frame, face)
            roi_mask = np.maximum(roi_mask, mask)

        # Blur the mask
        roi_mask = cv2.GaussianBlur(roi_mask, (21, 21), 20)

        # Normalize the mask between 0 and 1
        roi_mask = roi_mask / 255

        masks.append(roi_mask)
    return masks

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
frames, meta = dataset.get_video_rgb('Proband16', '101_natural_lighting', 10)
masks = get_face_masks(frames)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].imshow(frames[0])

blurry_frame = np.uint8(frames[0] * masks[0])

ax[1].imshow(blurry_frame)
plt.show()

In [ ]:
def store_video_with_mask(frames, masks, filename, fps):
    shape = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'FFV1')
    out = cv2.VideoWriter(filename, fourcc, fps, (shape[1], shape[0]))

    for idx, mask in enumerate(masks):
        # Apply the mask to the frame
        frame = np.uint8(frames[idx] * mask)

        # Convert the frame to BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Write the frame
        out.write(frame)

    out.release()

In [ ]:
import os
import subprocess
from respiration.dataset import VitalCamSet
from tqdm.auto import tqdm

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

for (subject, setting) in tqdm(scenarios):
    print(f"subject: {subject}")

    destination = os.path.join(dataset.data_path, subject, '302_face_only_blurry')

    # Copy the unisens data to the new scenario
    source = os.path.join(dataset.data_path, subject, setting, 'synced_Logitech HD Pro Webcam C920')
    os.makedirs(destination, exist_ok=True)
    subprocess.run(["cp", "-r", source, destination])

    frames, meta = dataset.get_video_rgb(subject, setting)
    masks = get_face_masks(frames)
    video_path = os.path.join(destination, 'Logitech HD Pro Webcam C920.avi')
    store_video_with_mask(frames, masks, video_path, meta.fps)